##

In [ ]:
%load_ext autoreload
%autoreload 2
import plotnine as gg

from kth_sr.utils import get_df_by_downloaded_folder

folder = "../data/download_df_audio/"
new_folder = "../data/filtered_celebs_data/"
df = get_df_by_downloaded_folder(folder)
df.head()

In [ ]:
df[df["sample_number"] == 0].count()

In [ ]:
# delete ids with less than 10 samples

df = df.groupby("speaker").filter(lambda x: len(x) >= 10)
df["speaker"].nunique()
# we filter out 1 speaker

In [ ]:
# show shortest samples for each speaker
shortest_samples = df.groupby("speaker")["duration_s"].min()
shortest_samples

In [ ]:
(
    gg.ggplot(shortest_samples.reset_index(), gg.aes(x="duration_s"))
    + gg.geom_histogram(bins=30)
    # labels and title
    + gg.labs(
        title="Shortest samples duration",
        x="Duration in seconds",
        y="Count",
    )
)

In [ ]:
# filter ids with lowest duration lower than 10 seconds

df_filtered = df.groupby("speaker").filter(lambda x: x["duration_s"].min() >= 10)
df_filtered["speaker"].nunique()
# we have 232 speakers left.

In [ ]:
# sort unique ids by shortest duration
shortest_samples_filtered = df_filtered.groupby("speaker")["duration_s"].min()
shortest_samples_filtered = shortest_samples_filtered.sort_values(ascending=False)[:200]
shortest_samples_filtered

df_filtered = df_filtered[df_filtered["speaker"].isin(shortest_samples_filtered.index)]
df_filtered["speaker"].nunique()

In [ ]:
# get shortest samples
df_filtered.sort_values(by="duration_s", ascending=True).head()

In [ ]:
# copy files to new folder
import shutil

for index, row in df_filtered.iterrows():
    # copy file
    shutil.copy(folder + row["file_name"], new_folder + row["file_name"])
    print(row["file_name"])

In [ ]:
# check if files are copied
df_filtered_check = get_df_by_downloaded_folder(new_folder)
df_filtered_check

In [ ]:
# check if all ids have 10 samples
df_filtered_check.groupby("speaker")["sample_number"].nunique().nunique()